# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [ ]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
from torch import optim
from torch.autograd import Variable
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10
learning_rate = 1e-2

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [ ]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [ ]:
class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self, in_dim, n_hidden_1, n_hidden_2, out_dim):
        super(SimpleNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(in_dim, n_hidden_1),
            nn.BatchNorm1d(n_hidden_1), nn.ReLU(True))
        self.layer2 = nn.Sequential(
            nn.Linear(n_hidden_1, n_hidden_2),
            nn.BatchNorm1d(n_hidden_2), nn.ReLU(True))
        self.layer3 = nn.Sequential(nn.Linear(n_hidden_2, out_dim))

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x




    
model = SimpleNet(28*28,100,10,10)

# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)



Next, we can start to train and evaluate!

In [ ]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        
        images = images.view(images.size(0), -1)
        images = Variable(images)
        labels = Variable(labels)
        out = model(images)
        loss = criterion(out, labels)
        print_loss = loss.data.item()
    
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        '''if epoch%50 == 0:
            print('epoch: {}, loss: {:.4}'.format(epoch, loss.data.item()))    '''  
# evaluate
# TODO:calculate the accuracy using traning and testing dataset
model.eval()
eval_loss1 = 0
eval_acc1 = 0
for images, labels in tqdm(test_loader):
    images = images.view(images.size(0), -1)  
    out = model(images)
    loss = criterion(out, labels)
    eval_loss1 += loss.data.item()*labels.size(0)
    _, pred = torch.max(out, 1)
    num_correct = (pred == labels).sum()
    eval_acc1 += num_correct.item()

eval_loss2 = 0
eval_acc2 = 0
for images, labels in tqdm(train_loader):
    images = images.view(images.size(0), -1)  
    out = model(images)
    loss = criterion(out, labels)
    eval_loss2 += loss.data.item()*labels.size(0)
    _, pred = torch.max(out, 1)
    num_correct = (pred == labels).sum()
    eval_acc2 += num_correct.item()

    
    


#### Q5:
Please print the training and testing accuracy.

In [ ]:
print('test accuracy: {:.6f}'.format(
    eval_acc1 / (len(test_dataset))
))
print('train accuracy: {:.6f}'.format(
    eval_acc2 / (len(train_dataset))
))
